We are using the dataset lisa_tiny and yolo v5

In [ ]:
# setting up yolo v5

!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -U -r requirements.txt


In [6]:
# setting up dataset

import os
import random
import shutil
import pandas as pd
from PIL import Image

# define vars
image_folder = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny'
annotation_file = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/annotations.csv'

# Create directories for train and val if they don't exist
train_images = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/images/train'
val_images = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/images/val'
train_labels = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/labels/train/'
val_labels = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/labels/val/'

os.makedirs(train_images, exist_ok=True)
os.makedirs(val_images, exist_ok=True)
os.makedirs(train_labels, exist_ok=True)
os.makedirs(val_labels, exist_ok=True)

# Load annotations
annotations = pd.read_csv(annotation_file)

# Get unique image filenames
image_files = annotations['filename'].unique()

# Shuffle the list of image files
random.shuffle(image_files)

# Split the dataset (80% for training, 20% for validation)
train_size = int(0.8 * len(image_files))  # 80% for training
train_files = image_files[:train_size]
val_files = image_files[train_size:]

# Move images to the appropriate directories
# for file in train_files:
#     shutil.move(os.path.join(image_folder, file), os.path.join(train_images, file))
# for file in val_files:
#     shutil.move(os.path.join(image_folder, file), os.path.join(val_images, file))

print("moved all files to respective folders")

# Map class names to class IDs
class_map = {
    'stop': 0,
    'yield': 1,
    'yieldAhead': 2,
    'merge': 3,
    'signalAhead': 4,
    'pedestrianCrossing': 5,
    'keepRight': 6,
    'speedLimit35': 7,
    'speedLimit25': 8
}

# since images are already in their respective folders
train_files = os.listdir(train_images)
val_files = os.listdir(val_images)

# Process each image in the training set
for file in train_files:
    # Get the corresponding annotations for the image
    image_annotations = annotations[annotations['filename'] == file]

    # Get image size (to normalize bounding box coordinates)
    image_path = os.path.join(image_folder, file)
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Create the corresponding label file for YOLO
    label_file = os.path.join(train_labels, file.replace('.png', '.txt'))

    with open(label_file, 'w') as f:
        for _, row in image_annotations.iterrows():
            # Get bounding box coordinates
            x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
            class_name = row['class']
            class_id = class_map[class_name]  # Map class name to ID

            # Calculate YOLO format: x_center, y_center, width, height (normalized)
            x_center = (x1 + x2) / 2 / image_width
            y_center = (y1 + y2) / 2 / image_height
            width = (x2 - x1) / image_width
            height = (y2 - y1) / image_height

            # Write to label file
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process each image in the validation set
for file in val_files:
    # Get the corresponding annotations for the image
    image_annotations = annotations[annotations['filename'] == file]

    # Get image size (to normalize bounding box coordinates)
    image_path = os.path.join(image_folder, file)
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Create the corresponding label file for YOLO
    label_file = os.path.join(val_labels, file.replace('.png', '.txt'))

    with open(label_file, 'w') as f:
        for _, row in image_annotations.iterrows():
            # Get bounding box coordinates
            x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
            class_name = row['class']
            class_id = class_map[class_name]  # Map class name to ID

            # Calculate YOLO format: x_center, y_center, width, height (normalized)
            x_center = (x1 + x2) / 2 / image_width
            y_center = (y1 + y2) / 2 / image_height
            width = (x2 - x1) / image_width
            height = (y2 - y1) / image_height

            # Write to label file
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("Processed and created YOLO label files for all images")

# Create the YAML file for YOLOv5
with open('/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/traffic_sign.yaml', 'w') as f:
    f.write("""
    train: /content/drive/My Drive/traffic_sign_recognition/lisa_tiny/images/train
    val: /content/drive/My Drive/traffic_sign_recognition/lisa_tiny/images/val
    nc: 9  # Number of classes
    names: ['stop', 'yield', 'yieldAhead', 'merge', 'signalAhead', 'pedestrianCrossing', 'keepRight', 'speedLimit35', 'speedLimit25']
    """)


moved all files to respective folders


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/sample_119.png'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
